In [1]:
from os import listdir
import tqdm
import zipfile
import pandas as pd 
import numpy as np
pd.options.mode.chained_assignment = None  # default='warn'

unzip_files = False

In [2]:
path = "C:/Users/Marie/Desktop/Bike Sharing/Full Data/"

In [3]:
compressed_files = listdir(path + 'Compressed/')

In [4]:
if unzip_files:    
    for file in tqdm.tqdm(compressed_files):
        zip_file = zipfile.ZipFile(path + 'Compressed/' + file, 'r')
        zip_file.extractall(path)
        zip_file.close()

In [5]:
full_files = listdir(path)
full_files = [f for f in full_files if f[-3:] == 'csv']

In [6]:
full_data = {}

cleaning_all = False

for f in tqdm.tqdm(full_files):
    if f[-3:] == 'csv':
        df_temp = pd.read_csv(path+f)
        # df_temp = df_temp.dropna()
        df_temp = df_temp.drop(df_temp.columns[0], axis = 1)
        cols = list(df_temp.columns)
        cols[0] = 'StartDate'
        cols[-1] = 'MemberType'
        cols[-2] = 'BikeNumber'
        
        cols = [str(col).lower().replace(' ', '') for col in cols]
        
        df_temp.columns = cols
        if cleaning_all == True:
            if len(cols) == 6:
                df_temp['endstationnumber'] = df_temp['endstation'].apply(lambda x: str(x).split(' (')[-1][:-1])
                #df_temp = df_temp[df_temp.endstationnumber.apply(lambda x: len(str(x)))==5]
                df_temp['endstation'] = df_temp['endstation'].apply(lambda x: str(x).split(' (')[0])

                df_temp['startstationnumber'] = df_temp['startstation'].apply(lambda x: str(x).split(' (')[-1][:-1])
                #df_temp = df_temp[df_temp.startstationnumber.apply(lambda x: len(str(x)))==5]
                df_temp['startstation'] = df_temp['startstation'].apply(lambda x: str(x).split(' (')[0])
        else :
            df_temp = df_temp[['startdate', 'membertype']]
            
            
        df_temp['casual'] = df_temp['membertype'].apply(lambda x: str(x).lower()=='casual')*1
        df_temp['registered'] = 1- df_temp['casual']
        df_temp['cnt'] = 1
        df_temp = df_temp.drop('membertype', axis =1)
        
        
        full_data[f] = {
            'data':df_temp,
            'size':df_temp.shape,
            'columns':df_temp.columns,
            'types':df_temp.dtypes
        }

100%|██████████████████████████████████████████████████████████████████████████████████| 26/26 [00:44<00:00,  1.71s/it]


In [39]:
key = list(full_data.keys())[0]

df_daily = full_data[key]['data']

df_daily['startdate'] = pd.to_datetime(df_daily['startdate'])
df_daily['year'] = df_daily['startdate'].apply(lambda x: x.year)
df_daily['month'] = df_daily['startdate'].apply(lambda x: x.month)
df_daily['day'] = df_daily['startdate'].apply(lambda x: x.day)
df_daily['wkday'] = df_daily['startdate'].apply(lambda x: x.weekday())
df_daily['hour'] = df_daily['startdate'].apply(lambda x: x.hour)
df_daily = df_daily.drop('startdate', axis = 1) 

df_daily = df_daily.groupby(['year', 'month', 'day', 'wkday', 'hour']).sum()

for key in tqdm.tqdm(list(full_data.keys())[1:]):
    
    df_by_day = full_data[key]['data']

    df_by_day['startdate'] = pd.to_datetime(df_by_day['startdate'])
    df_by_day['year'] = df_by_day['startdate'].apply(lambda x: x.year)
    df_by_day['month'] = df_by_day['startdate'].apply(lambda x: x.month)
    df_by_day['day'] = df_by_day['startdate'].apply(lambda x: x.day)
    df_by_day['wkday'] = df_by_day['startdate'].apply(lambda x: x.weekday())
    df_by_day['hour'] = df_by_day['startdate'].apply(lambda x: x.hour)
    df_by_day = df_by_day.drop('startdate', axis = 1)
    
    df_daily = pd.concat([df_daily, df_by_day.groupby(['year', 'month', 'day', 'wkday', 'hour']).sum()], axis = 0)

100%|██████████████████████████████████████████████████████████████████████████████████| 24/24 [04:46<00:00, 11.93s/it]


In [40]:
df_daily.reset_index().to_csv(path + 'Cleaned Data/full_hourly_data.csv', index = False)

In [ ]:
key = list(full_data.keys())[0]

df_end_station = full_data[key]['data'][['endstation', 'endstationnumber']].drop_duplicates()

for key in list(full_data.keys())[1:]:
    df_end_station = pd.concat([df_end_station, full_data[key]['data'][['endstation', 'endstationnumber']].drop_duplicates()], axis = 0)
    
df_end_station['temp'] = df_end_station['endstation'].apply(lambda x: str(x).split(' (')[-1][:-1])
df_end_station['endstation'] = df_end_station['endstation'].apply(lambda x: str(x).split(' (')[0])

df_end_station['endstationnumber'][df_end_station['endstationnumber']==0] = df_end_station['temp'][df_end_station['endstationnumber']==0]

df_end_station = df_end_station.drop('temp', axis = 1).drop_duplicates()

print(len(pd.unique(df_end_station.endstationnumber)))

df_end_station.columns = ['station', 'stationNumber']

In [ ]:
key = list(full_data.keys())[0]

df_start_station = full_data[key]['data'][['startstation', 'startstationnumber']].drop_duplicates()

for key in list(full_data.keys())[1:]:
    df_start_station = pd.concat([df_start_station, full_data[key]['data'][['startstation', 'startstationnumber']].drop_duplicates()], axis = 0)
    
df_start_station['temp'] = df_start_station['startstation'].apply(lambda x: str(x).split(' (')[-1][:-1])
df_start_station['startstation'] = df_start_station['startstation'].apply(lambda x: str(x).split(' (')[0])

df_start_station['startstationnumber'][df_start_station['startstationnumber']==0] = df_start_station['temp'][df_start_station['startstationnumber']==0]

df_start_station = df_start_station.drop('temp', axis = 1).drop_duplicates()

print(len(pd.unique(df_start_station.startstationnumber)))

df_start_station.columns = ['station', 'stationNumber']

In [ ]:
df_stations = pd.concat([df_start_station, df_end_station], axis = 0)

In [ ]:
df_stations = df_stations.drop_duplicates()

In [ ]:
full_data[key]['data']

In [ ]:
df_stations = df_stations[(df_stations.stationNumber.apply(lambda x: len(str(x))))==5]

In [ ]:
df_stations.stationNumber = df_stations.stationNumber.astype(int)

In [ ]:
df_stations.to_csv(path + 'Cleaned Data/stations.csv', index = False)

In [7]:
s = 0 
for key in full_data.keys():
    s +=full_data[key]['size'][0]
print(s)

14403639


In [8]:
len(full_data.keys())

26